# Practicing link prediction with PyG

## Using a 'custom' `decoder` function
base: https://github.com/pyg-team/pytorch_geometric/blob/master/examples/link_pred.py

In [1]:
import torch

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from torch_geometric.data import Data
from torch_geometric.utils import train_test_split_edges

from sklearn.metrics import roc_auc_score

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


In [3]:
transform = T.Compose([ # composite transform
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1) # 5% for validation, 10% for testing and the rest 85% for training
])

In [4]:
dataset = Planetoid(root='datasets/Cora', name='Cora', transform=transform)
dataset

Cora()

In [5]:
dataset_not_transformed = Planetoid(root='datasets/Cora', name='Cora')
dataset_not_transformed[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [6]:
dataset[0] # what are the `edge_label` and `edge_label_index`
# i think, the `RandomLinkSplit` adds one negative samples for every positive samples in each of the returning graphs
# `edge_index` holds the positive samples, used for message passing and encoding the graph structure
# `edge_label_index` holds the positive+negative samples, used for decoding and calculating the loss i.e. supervision

(Data(x=[2708, 1433], edge_index=[2, 8974], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[17948], edge_label_index=[2, 17948]),
 Data(x=[2708, 1433], edge_index=[2, 8974], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[1054], edge_label_index=[2, 1054]),
 Data(x=[2708, 1433], edge_index=[2, 9501], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[2110], edge_label_index=[2, 2110]))

In [7]:
train_data, val_data, test_data = dataset[0]
train_data

Data(x=[2708, 1433], edge_index=[2, 8974], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[17948], edge_label_index=[2, 17948])

In [8]:
train_data.edge_index == val_data.edge_index

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]], device='cuda:0')

In [9]:
train_data.is_undirected()

False

In [10]:
train_data.edge_label.unique(return_counts=True)

(tensor([0., 1.], device='cuda:0'), tensor([8974, 8974], device='cuda:0'))

In [11]:
train_data.edge_label_index

tensor([[1776, 1405, 1645,  ..., 1002, 1886,  616],
        [1623, 1053,  989,  ...,  200, 1752,  667]], device='cuda:0')

In [12]:
train_data.edge_index

tensor([[1776, 1405, 1645,  ...,  571, 2531, 2154],
        [1623, 1053,  989,  ..., 1570, 1026, 1224]], device='cuda:0')

In [13]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
    
    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    
    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=1) # sum(subject_nodes * object_nodes)
    
    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()
    
    
    def forward(self, data, neg_edge_index=None): 
        z = self.encode(data.x, data.edge_index)
        
        if not self.training:
            return self.decode(z, data.edge_label_index).view(-1)

        edge_label_index = torch.cat([
            data.edge_label_index, neg_edge_index
        ], dim=1)

        return self.decode(z, edge_label_index).view(-1)
    

In [14]:
model = GCN(dataset.num_features, 128, dataset.num_classes).to(device)
model

GCN(
  (conv1): GCNConv(1433, 128)
  (conv2): GCNConv(128, 7)
)

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [16]:
train_data.edge_label_index.shape

torch.Size([2, 17948])

In [17]:
def train():
    model.train()
    optimizer.zero_grad()

    z = model.encode(train_data.x, train_data.edge_index)
    # NOTE: encoding is done on the original data, not on the data with negative samples, which is stored in `edge_label_index`
    # and then before, decode, we calculate another new set of negative samples in each epoch and add them to the data
    # so, here, we are not using the negative samples for encoding, but we are using them for decoding
    # and we are calculating negative samples twice, once when loading data with `RandomLinkSplit` and once here witch `negative_sampling`
    # the first negative samples are always the same, but the second ones are different in each epoch
    

    # calculating a new round of negative sampling for every every epoch
    neg_edge_index = negative_sampling(
        edge_index = train_data.edge_index,
        num_nodes = train_data.num_nodes,
        num_neg_samples = train_data.edge_label_index.size(1),
        method='sparse'
    )
    # edge_index = torch.as_tensor([[0, 0, 1, 2],
    #                               [0, 1, 2, 3]])
    # negative_sampling(edge_index)
    # tensor([[3, 0, 0, 3],
    #   [2, 3, 2, 1]])
    # is not a bad idea? we are teaching the model that (3, 2) is a negative sample, but what if (3, 2) is a positive sample in the test set or in reality?
    # should we not corrupt the positive samples instead to create negative ones?
    # or is this how negative sampling works? we are not sure if the negative samples are actually negative samples or not?

    # concatenating the new neg samples with the existing samples
    edge_label_index = torch.cat([
        train_data.edge_label_index, 
        neg_edge_index
    ], dim=-1)


    # concatenating the labels(0s) for the new neg samples with the existing labels
    edge_label = torch.cat([
        train_data.edge_label, # 'main' labels
        train_data.edge_label.new_zeros(neg_edge_index.size(1)) # 0s for the new neg samples
    ], dim = 0)

    # NOTE: Negative sample are only used for decoding, not for encoding

    out = model.decode(z, edge_label_index).view(-1)
    
    loss = criterion(out, edge_label) # loss calculation
    loss.backward() # backpropagation
    optimizer.step() # weight update
    return loss
    

In [18]:
def train2():
    model.train()
    optimizer.zero_grad()

    neg_edge_index = negative_sampling(
        edge_index = train_data.edge_index,
        num_nodes = train_data.num_nodes,
        num_neg_samples = train_data.edge_label_index.size(1),
        method='sparse'
    )

    edge_label = torch.cat([
        train_data.edge_label, 
        torch.zeros(neg_edge_index.size(1), dtype=torch.float, device=device)
    ], dim=-1)

    out = model(train_data, neg_edge_index)

    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss    

In [19]:
@torch.no_grad()
def test(data: Data):
    model.eval() # set the model to evaluation mode
    z = model.encode(data.x, data.edge_index) # encode the data
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid() # decode the data
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy()) # calculate the roc auc score
    

In [20]:
@torch.no_grad()
def test2(data: Data):
    model.eval()
    out = model(data)
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [21]:
best_val_auc = final_test_auc = 0
for epoch in range(1, 10):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)
final_edge_index

Epoch: 001, Loss: 0.6932, Val: 0.7518, Test: 0.7491
Epoch: 002, Loss: 0.6969, Val: 0.7417, Test: 0.7287
Epoch: 003, Loss: 0.6934, Val: 0.6607, Test: 0.6945
Epoch: 004, Loss: 0.6932, Val: 0.6025, Test: 0.6352
Epoch: 005, Loss: 0.6933, Val: 0.4786, Test: 0.4984
Epoch: 006, Loss: 0.6934, Val: 0.4687, Test: 0.4788
Epoch: 007, Loss: 0.6934, Val: 0.4761, Test: 0.4773
Epoch: 008, Loss: 0.6934, Val: 0.4795, Test: 0.4815
Epoch: 009, Loss: 0.6934, Val: 0.4839, Test: 0.4784
Final Test: 0.7491


tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [   0,    1,    2,  ..., 2705, 2706, 2707]], device='cuda:0')

In [22]:
best_val_auc = final_test_auc = 0
for epoch in range(1, 10):
    loss = train2()
    val_auc = test2(val_data)
    test_auc = test2(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)
final_edge_index

Epoch: 001, Loss: 0.6933, Val: 0.5123, Test: 0.4744
Epoch: 002, Loss: 0.6933, Val: 0.5165, Test: 0.5072
Epoch: 003, Loss: 0.6932, Val: 0.5145, Test: 0.5091
Epoch: 004, Loss: 0.6932, Val: 0.5109, Test: 0.5175
Epoch: 005, Loss: 0.6932, Val: 0.5109, Test: 0.5191
Epoch: 006, Loss: 0.6932, Val: 0.5110, Test: 0.5191
Epoch: 007, Loss: 0.6932, Val: 0.5126, Test: 0.5192
Epoch: 008, Loss: 0.6932, Val: 0.5181, Test: 0.5192
Epoch: 009, Loss: 0.6932, Val: 0.4945, Test: 0.5210
Final Test: 0.5192


tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [   0,    1,    2,  ..., 2705, 2706, 2707]], device='cuda:0')

## Understanding the various 'utils' functions
mainly `nagative_sampling`, `RandomLinkSplit` and `train_test_split_edges`

In [23]:
x = torch.tensor([
    [1, 2, 3], 
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12],
    [13, 14, 15]
])
edge_index = torch.tensor([
    [0, 1, 2],
    [1, 2, 0]
])

data = Data(x=x, edge_index=edge_index)
data

Data(x=[5, 3], edge_index=[2, 3])

In [24]:
negative_sampling(edge_index)

tensor([[0, 1, 2],
        [2, 0, 1]])

In [25]:
negative_sampling(
    edge_index = edge_index,
    num_nodes = data.num_nodes, # 3
    num_neg_samples = 6,
)

tensor([[1, 4, 3, 3, 2, 4],
        [3, 0, 1, 0, 1, 3]])

In [26]:
train_data, val_data, test_data = T.RandomLinkSplit(num_val=0.05, num_test=0.1)(data)
[train_data, val_data, test_data]

[Data(x=[5, 3], edge_index=[2, 3], edge_label=[6], edge_label_index=[2, 6]),
 Data(x=[5, 3], edge_index=[2, 3], edge_label=[0], edge_label_index=[2, 0]),
 Data(x=[5, 3], edge_index=[2, 3], edge_label=[0], edge_label_index=[2, 0])]

In [27]:
train_test_split_edges(data.clone(), val_ratio=0.05, test_ratio=0.1) # in-place

d:\Installations\JupyterNotebook\Lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Data(x=[5, 3], val_pos_edge_index=[2, 0], test_pos_edge_index=[2, 0], train_pos_edge_index=[2, 4], train_neg_adj_mask=[5, 5], val_neg_edge_index=[2, 0], test_neg_edge_index=[2, 0])

In [28]:
data

Data(x=[5, 3], edge_index=[2, 3])

In [29]:
train_data.edge_label_index

tensor([[0, 1, 2, 1, 0, 3],
        [1, 2, 0, 3, 4, 0]])